# In this notebook, we will explore the effect of the ratio of f to p on the model

In [ ]:
from simulations.drosselschwab import simulate_drosselschwab
import results
import data
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing
import csv
import os
from pathlib import Path
from datetime import datetime
from scripts.parallel_sims import worker

In [ ]:
outdir = Path("../data/f_over_p").resolve()
outdir.mkdir(exist_ok=True)

In [ ]:
# Example parameter sweep: small demo grid. Replace with your actual sweep.
param_list = []
param_idx = 0
L = 100
steps = 1000

runs_per_param = 2

# vary p and f in a few combinations (one simulation per tuple)
p_values = [0.005, 0.01]
f_values = [0.0001, 0.0005, 0.001]

for p in p_values:
    for f in f_values:

        for run_idx in range(runs_per_param):
            param_list.append({'L': L, 'p': p, 'f': f, 'steps': steps, 'param_id': param_idx, 'run_id': run_idx})
        param_idx += 1

In [ ]:
max_workers = int(os.environ.get('MAX_WORKERS', multiprocessing.cpu_count()))
print(f"Running {len(param_list)} simulations with up to {max_workers} workers...")

results = []
with ProcessPoolExecutor(max_workers=max_workers) as exe:
    # pass outdir as first argument to worker
    futures = {exe.submit(worker, outdir, params): params for params in param_list}
    for fut in as_completed(futures):
        params = futures[fut]
        try:
            res = fut.result()
            print(f"Done: p={res['p']}, f={res['f']}, fires={res['num_fires']}, mean={res['mean_size']:.2f}, max={res['max_size']}")
            results.append(res)
        except Exception as e:
            print(f"Error for params {params}: {e}")

# Write a summary CSV
summary_file = outdir / f"summary_{datetime.now().strftime('%Y%m%dT%H%M%SZ')}.csv"
keys = ['L', 'p', 'f', 'steps', 'param_id', 'run_id', 'num_fires', 'mean_size', 'max_size', 'remaining_trees', 'raw_file']
with open(summary_file, 'w', newline='') as fh:
    writer = csv.DictWriter(fh, keys)
    writer.writeheader()
    for r in results:
        writer.writerow({k: r.get(k, '') for k in keys})

print(f"Summary written to {summary_file}")
